# Inverse problem
known: all data  
unknown: k8, ka, k5, h5  
results record: https://outlookuga-my.sharepoint.com/:x:/r/personal/yq88347_uga_edu/_layouts/15/Doc.aspx?sourcedoc=%7Be6449b96-7895-43ff-a1e4-42fc2118f900%7D&action=editnew

In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

import tensorflow as tf
import models_tf as models
import time

In [2]:
# data = sio.loadmat("../test.mat")
data = sio.loadmat("../../data/data_2k_points.mat")
c0 = data["IC"]
t = data["t"]
c_ref = data["y"]

t_train = t

scale = np.array(
    [10, 1, 100, 100, 0.0001, 10, 0.1, 1000]
).reshape([-1])
c_ref = c_ref / scale
c0 = c0 / scale

In [3]:
np.max(t)

100.0

In [4]:
len(t)

2000

In [5]:
np.max(data["y"], axis=0)

array([2.99999999e+01, 3.54115121e+00, 8.57923329e+02, 1.00000000e+03,
       1.13698673e-03, 3.83874944e+01, 8.68819961e-01, 2.16923077e+03])

In [6]:
model = models.PINN(
    c0=c0,
    output_dim=8,
    scale=scale,
    units=100, 
    activation=tf.tanh,
    eps=1e-5,
)

eps:  tf.Tensor(1e-05, shape=(), dtype=float32)


In [7]:
# t_ode = t_train[::10]
# t_u = t_train
# u = c_ref
t_ode = t_train[:201]
t_u = t_train
u = c_ref
st = time.time()
# loss, ode_loss, data_loss, min_loss, h5_list = model.train(t_ode, t_u, u, niter=200000)
loss = model.train(t_ode, t_u, u, niter=500000)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

0 29.711248 2618674.2 3.524505 261625050000.0 24.236437 1.0009986
1000 0.19583985 53.489895 0.19530496 3668576.8 0.16804127 0.48353434
2000 0.10024772 51.195557 0.09973576 345206.28 0.47743493 0.23394519
3000 0.065049164 140.97691 0.063639395 186102.97 1.3911589 0.13087806
4000 0.052211665 148.89664 0.0507227 152959.86 1.4736704 0.07709957
5000 0.047354836 121.42343 0.0461406 172206.6 1.1970136 0.045897588
6000 0.045560524 106.67212 0.044493802 171390.47 1.0495821 0.028383777
7000 0.04510822 112.6215 0.043982007 1256162.6 1.0005988 0.017957143
8000 0.04473606 100.330376 0.04373276 161181.89 0.9871856 0.011524235
9000 0.044627286 100.39462 0.04362334 182941.19 0.9856521 0.0074784085
10000 0.050851945 677.8055 0.04407389 57022580.0 1.0757971 0.0048866556
11000 0.04671605 297.89767 0.043737072 19245022.0 1.0544746 0.00321027
12000 0.046119127 276.29248 0.043356203 17343966.0 1.0285285 0.002125621
13000 0.044363566 99.983536 0.04336373 2065493.4 0.793286 0.0014397547
14000 0.043885145 88.4

Execution time:  
2091.38174700737  
2203.2176880836487  
2342.4667887687683

In [9]:
(2091.4 + 2203.2 + 2342.5) / 3

2212.366666666667

In [8]:
min_loss

NameError: name 'min_loss' is not defined

In [ ]:
model.restore()
c_pred = model.call(
    tf.constant(t_train, tf.float32),
)

In [ ]:
L2 = np.sqrt(np.sum((c_pred - c_ref) ** 2, axis=0) / np.sum(c_ref ** 2, axis=0))

In [ ]:
L2

In [ ]:
# ka = 1.2
print(tf.math.exp(model.log_ka))

In [ ]:
# k8 = 0.00001
print(tf.math.exp(model.log_k8))

In [ ]:
# k5 = 0.17
print(tf.math.exp(model.log_k5))

In [ ]:
# h5 = 0.31
print(tf.math.exp(model.log_h5))

In [ ]:
plt.figure(dpi=200)
plt.plot(h5_list[::1000], "-", label="reference")
# plt.plot(np.log(loss)[::1000], "-", label="reference")
plt.show()

In [ ]:
plt.figure(dpi=200)
plt.plot(t_train, c_ref, "-", label="reference")
# plt.plot(t_train, y_train, '.', label="train")
plt.plot(t_train, c_pred, "--", label="prediction")
plt.xlabel("min")
plt.legend()
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.plot(np.log10(loss)[::1000], "b--", label="total loss", alpha=0.5)
plt.plot(np.log10(data_loss)[::1000], "r--", label="data loss", alpha=0.5)
plt.plot(np.log10(ode_loss)[::1000], "g-", label="ODE loss", alpha=1)
plt.xlabel('training epochs ($\\times$ 1000)')
plt.ylabel('$\\log$ loss')
plt.legend(fontsize=8)
plt.show()

In [ ]:
# plt.figure(dpi=100)
plt.figure(figsize=(12, 6), dpi=100)
plt.subplot(2, 4, 1)
plt.plot(t_train, c_ref[:, 0:1], "-", label="reference")
plt.plot(t_train, c_pred[:, 0:1], "--", label="prediction")
plt.xlabel("min")
plt.legend()

# plt.figure(dpi=100)
plt.subplot(2, 4, 2)
plt.plot(t_train, c_ref[:, 1:2], "-", label="reference")
plt.plot(t_train, c_pred[:, 1:2], "--", label="prediction")
plt.xlabel("min")
# plt.legend()

# plt.figure(dpi=100)
plt.subplot(2, 4, 3)
plt.plot(t_train, c_ref[:, 2:3], "-", label="reference")
plt.plot(t_train, c_pred[:, 2:3], "--", label="prediction")
plt.xlabel("min")
# plt.legend()

# plt.figure(dpi=100)
plt.subplot(2, 4, 4)
plt.plot(t_train, c_ref[:, 3:4], "-", label="reference")
plt.plot(t_train, c_pred[:, 3:4], "--", label="prediction")
plt.xlabel("min")
# plt.legend()

# plt.figure(dpi=100)
plt.subplot(2, 4, 5)
plt.plot(t_train, c_ref[:, 4:5], "-", label="reference")
plt.plot(t_train, c_pred[:, 4:5], "--", label="prediction")
plt.xlabel("min")
# plt.legend()

# plt.figure(dpi=100)
plt.subplot(2, 4, 6)
plt.plot(t_train, c_ref[:, 5:6], "-", label="reference")
plt.plot(t_train, c_pred[:, 5:6], "--", label="prediction")
plt.xlabel("min")
# plt.legend()

# plt.figure(dpi=100)
plt.subplot(2, 4, 7)
plt.plot(t_train, c_ref[:, 6:7], "-", label="reference")
plt.plot(t_train, c_pred[:, 6:7], "--", label="prediction")
plt.xlabel("min")
# plt.legend()

# plt.figure(dpi=100)
plt.subplot(2, 4, 8)
plt.plot(t_train, c_ref[:, 7:8], "-", label="reference")
plt.plot(t_train, c_pred[:, 7:8], "--", label="prediction")
plt.xlabel("min")
# plt.legend()

plt.suptitle("Results for data fitting")
plt.show()

In [ ]:
plt.figure(dpi=100)
plt.plot(t_train, c_ref[:, 0:1], "-", label="reference")
plt.plot(t_train, c_pred[:, 0:1], "--", label="prediction")
plt.legend()
plt.show()

plt.figure(dpi=100)
plt.plot(t_train, c_ref[:, 1:2], "-", label="reference")
plt.plot(t_train, c_pred[:, 1:2], "--", label="prediction")
plt.legend()
plt.show()

plt.figure(dpi=100)
plt.plot(t_train, c_ref[:, 2:3], "-", label="reference")
plt.plot(t_train, c_pred[:, 2:3], "--", label="prediction")
plt.legend()
plt.show()

plt.figure(dpi=100)
plt.plot(t_train, c_ref[:, 3:4], "-", label="reference")
plt.plot(t_train, c_pred[:, 3:4], "--", label="prediction")
plt.legend()
plt.show()

plt.figure(dpi=100)
plt.plot(t_train, c_ref[:, 4:5], "-", label="reference")
plt.plot(t_train, c_pred[:, 4:5], "--", label="prediction")
plt.legend()
plt.show()

plt.figure(dpi=100)
plt.plot(t_train, c_ref[:, 5:6], "-", label="reference")
plt.plot(t_train, c_pred[:, 5:6], "--", label="prediction")
plt.legend()
plt.show()

plt.figure(dpi=100)
plt.plot(t_train, c_ref[:, 6:7], "-", label="reference")
plt.plot(t_train, c_pred[:, 6:7], "--", label="prediction")
plt.legend()
plt.show()

plt.figure(dpi=100)
plt.plot(t_train, c_ref[:, 7:8], "-", label="reference")
plt.plot(t_train, c_pred[:, 7:8], "--", label="prediction")
plt.legend()
plt.show()